# 📊 **01 — Exploratory Data Analysis (EDA)**

**Notebook Purpose:**
Perform a *light, structured exploration* of the Kaggle **Detect Reversal Points in U.S. Equities** dataset. This notebook focuses on understanding distributions, spotting anomalies, mapping feature groups, and laying the groundwork for preprocessing and feature engineering.

---

**Competition:** *Detect Reversal Points in US Equities*
**Deadline:** December 31, 2025
**Repository:** `Kaggle-Detect-Reversal-Points-in-US-Equities`
**Author:** Brice Nelson

---

**Notebook Date Created:** 2025-11-29
**Notebook Last Updated:** 2025-11-29

---

## 🧭 **Goals of This Notebook**

- Load raw Kaggle training and test data (`/data/raw/`)
- Inspect dataset shape, dtypes, and missing values
- Explore Signal Descriptor columns
- Examine target class distribution (0–3)
- Identify time-related patterns and potential leakage risks
- Save EDA plots into `/figures/eda/`
- Produce initial notes for feature engineering

---

## 📂 **References**

- Project Plan: `docs/00_overview/01_reversal_points_project_plan.md`
- Folder Explanations: `docs/01_architecture/02_folder_explanations.md`
- Project Structure: `docs/01_architecture/01_project_structure.md`


In [5]:
# import libraries

import os
import sys
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.data.eda_utils import get_prefix_counts

# configurations
sns.set_theme(style="darkgrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

plt.style.use("seaborn-v0_8")

### Load Raw Data
- Load via Duckdb
- Create a connection
- Load the training and test datasets into dataframes


In [6]:
# Create duckdb connection

conn = duckdb.connect()

# Create duckdb dataframe

train_df = conn.execute("""
                        SELECT * FROM
                            read_csv_auto('../data/raw/new_competition_data/train.csv',
                            max_line_size=5000000)""").df()
test_df = conn.execute("""
                       SELECT * FROM
                           read_csv_auto('../data/raw/new_competition_data/test.csv',
                           max_line_size=5000000)""").df()

print('Train dataframe created.')
print('Test dataframe created.')

Train dataframe created.
Test dataframe created.


## Explore Descriptive Statistics

In [7]:
# Shape

print('Train dataframe shape:', train_df.shape)
print('Test dataframe shape:', test_df.shape)


Train dataframe shape: (2683, 68507)
Test dataframe shape: (1151, 68506)


In [8]:
# Train df info

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2683 entries, 0 to 2682
Columns: 68507 entries, train_id to class_label
dtypes: bool(68499), datetime64[us](1), float64(4), int64(1), object(2)
memory usage: 175.4+ MB


In [9]:
# Test df info

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Columns: 68506 entries, id to zone_99.5
dtypes: bool(68499), datetime64[us](1), float64(4), int64(1), object(1)
memory usage: 75.3+ MB


## 🦆 Why Use DuckDB for This EDA?

This dataset is extremely *wide* — more than **68,000 columns** of boolean and numeric features.
Instead of forcing pandas to scan tens of thousands of columns, DuckDB lets us work directly with the dataset **metadata**, making EDA faster and more efficient.

### 🔑 Key Advantages
- **Instant schema inspection** via `pragma_table_info()`
- **Fast prefix-based grouping** to discover feature families
- **Easy train/test column alignment checks**
- **Vectorized SQL operations** that scale better than Python loops

In short:
**DuckDB handles wide datasets with ease, giving us a cleaner, faster EDA workflow.**


## 🔍 DuckDB + Pandas Data Workflow (Short Summary)

For this project, we use a hybrid approach that combines the speed of **DuckDB** with the flexibility of **pandas**:

1. **Load CSVs with DuckDB** using `read_csv_auto()` for fast ingestion of 68,000+ columns.
2. **Convert results to pandas** via `.df()` so the data is ready for modeling (LightGBM, sklearn, etc.).
3. **Register the pandas DataFrames back into DuckDB** so we can run fast SQL queries for:
   - prefix-based feature grouping
   - schema comparisons (train vs. test)
   - constant-column detection
   - metadata inspection via `pragma_table_info()`

This workflow gives us **fast schema exploration** with DuckDB and **full ML compatibility** with pandas — the best setup for extremely wide datasets.


In [10]:
# Create instance for grouping helper function

prefix_summary = get_prefix_counts(train_df)
prefix_summary.head(20)

,prefix,count
0,occurs,34220
1,happens,34220
2,cross,26
3,zone,13
4,trending,10
5,peaks,5
6,troughs,5
7,sm,2
8,ticker,1
9,t,1


In [11]:
# Register for SQL use
duckdb.register('train', train_df)
duckdb.register('test', test_df)

# Convert column names into pandas DataFrames
col_df = pd.DataFrame({'column_name': train_df.columns})
duckdb.register('cols', col_df)

# Extract prefixes and count them
duckdb.query("""
    SELECT
        regexp_extract(column_name, '^[^_]+') AS prefix,
        COUNT(*) AS count
    FROM cols
    GROUP BY prefix
    ORDER BY count DESC
""").df()



,prefix,count
0,occurs,34220
1,happens,34220
2,cross,26
3,zone,13
4,trending,10
5,peaks,5
6,troughs,5
7,sm,2
8,ticker,1
9,t,1


In [12]:
# Train Descriptive Statistics
train_df.describe()

,train_id,t,momentum,ratio,sm_momentum,sm_ratio
count,2683.00000,2683,2683.000000,2683.000000,2683.000000,2683.000000
mean,1341.00000,2024-07-10 12:12:04.562057,100.000497,99.991561,99.999178,99.974807
min,0.00000,2023-04-03 00:00:00,91.807787,88.274743,97.687889,97.525819
25%,670.50000,2023-11-24 00:00:00,99.541766,99.388169,99.530776,99.646269
50%,1341.00000,2024-07-10 00:00:00,99.986933,99.983864,99.976695,99.981217
75%,2011.50000,2025-02-26 12:00:00,100.486041,100.652586,100.443431,100.284841
max,2682.00000,2025-10-17 00:00:00,104.532756,107.908723,103.115449,101.962479
std,774.65971,NaN,0.857252,1.237986,0.741165,0.503400


In [13]:
# train info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2683 entries, 0 to 2682
Columns: 68507 entries, train_id to class_label
dtypes: bool(68499), datetime64[us](1), float64(4), int64(1), object(2)
memory usage: 175.4+ MB


In [14]:
# Identify object columns

train_df.select_dtypes(include=['object']).columns.tolist()




['ticker_id', 'class_label']

In [15]:
train_df['class_label'].head()

0    None
1       H
2    None
3    None
4    None
Name: class_label, dtype: object

## Dataset Limitations
Due to the extreme column dimensionality (~68k features), all data previews explicitly select a small subset of semantic columns rather than using `SELECT *`.

In [19]:
# obtain head from data with duckdb

duckdb.sql("""
    SELECT
    train_id,
    ticker_id,
    t,
    class_label
    FROM train
    LIMIT 5;
""")



┌──────────┬───────────┬─────────────────────┬─────────────┐
│ train_id │ ticker_id │          t          │ class_label │
│  int64   │  varchar  │      timestamp      │   varchar   │
├──────────┼───────────┼─────────────────────┼─────────────┤
│        0 │ 001       │ 2024-01-10 00:00:00 │ None        │
│        1 │ 004       │ 2025-06-06 00:00:00 │ H           │
│        2 │ 002       │ 2024-07-29 00:00:00 │ None        │
│        3 │ 001       │ 2024-11-11 00:00:00 │ None        │
│        4 │ 003       │ 2025-01-15 00:00:00 │ None        │
└──────────┴───────────┴─────────────────────┴─────────────┘

In [20]:
# count booleans

duckdb.sql("""
SELECT
    COUNT(*) AS n_rows,
    SUM(CASE WHEN class_label = 'peaks' THEN 1 ELSE 0 END) AS peaks
FROM train;
    """)


┌────────┬────────┐
│ n_rows │ peaks  │
│ int64  │ int128 │
├────────┼────────┤
│   2683 │      0 │
└────────┴────────┘

### Target Variable Overview

The target variable `class_label` is a categorical event-based label with three valid values:

- `H` — Swing High
- `L` — Swing Low
- `None` — No reversal event

As confirmed by the competition documentation and the provided `sample_submission.csv`, predictions must take exactly one of these three values. The label distribution is highly imbalanced, with `None` dominating, which is expected given the sparse nature of reversal events in financial time series. This motivates the use of Macro F1-score as the primary evaluation metric.
